# 노트북1: ProtGPT2로 펩타이드 후보 생성 및 FASTA 파일 저장

In [ ]:

!pip install --upgrade transformers sentencepiece

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("nferruz/ProtGPT2")
model = AutoModelForCausalLM.from_pretrained("nferruz/ProtGPT2").to("cuda" if torch.cuda.is_available() else "cpu")

N = 5  # 생성할 후보 수 (원하는 만큼 변경)
peptides = []

for _ in range(N):
    input_ids = tokenizer("generate:", return_tensors="pt").input_ids.to(model.device)
    output = model.generate(input_ids, max_length=30, num_return_sequences=1, do_sample=True, top_k=950, top_p=0.96)
    sequence = tokenizer.decode(output[0], skip_special_tokens=True).replace("generate:", "").strip()
    peptides.append(sequence)

# 펩타이드 FASTA 파일 저장
for i, pep in enumerate(peptides):
    with open(f"peptide_{i}.fasta", "w") as f:
        f.write(f">pep{i}\n{pep}\n")

print("✅ 생성된 펩타이드 후보:")
for i, seq in enumerate(peptides):
    print(f"[{i}] {seq}")
